In [26]:
import pandas as pd
import numpy as np
from pathlib import Path

In [27]:
# hugoferq
work =  Path(r'D:\\Trabajo\AITeacherAllocation')
# Mariadelaguarda
#work =  Path(r'D:\\Trabajo\AITeacherAllocation')

In [28]:
df = pd.read_csv(work/r'Results\\Base consolidada.csv')

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_20432\3792860488.py:1: DtypeWarning: Columns (1,122) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(work/r'Results\\Base consolidada.csv')


# Target population

The goal is predict teacher allocation in this population target:

- Educational stage
    - Kindergarten 
    - Primary school

- Teachers:
    - Teachers
    - Education Assistant

- Type of school 
    - Public managed by Central Goverment

In [29]:
# Relevant rows
df = df.loc[df['niv_mod']!='F0'] # Educational stage
df = df.dropna(subset=['region']) # Drop missing in region
df = df.loc[(df['gestion']==1) & (df['ges_dep']=='A1')] # Type of school
df = df.loc[df['estado']==1] # Active in 2022

In [30]:
# Repair educational stage
df.loc[df['d_niv_mod']=='Inicial - Jard¡n','d_niv_mod'] = 'Inicial - Jardin'
df.loc[df['d_niv_mod']=='Inicial - Cuna-jard¡n','d_niv_mod'] = 'Inicial - Cuna-jardin'
# Drop few observations 
df = df.loc[df['d_niv_mod']!='Inicial - Cuna'] # 39 rows deleted
# Drop no relevant information in target variable
df = df.loc[df['secciones_necesarias']!=0] #720 rows deleted

In [31]:
# Drop irrelant columns
df = df.drop(columns=['Unnamed: 0.1','usuario_minedu','bolsa_nexus','bolsa_sira',
'dir_req', 'sub_dir_req','jer_req','fisica_req','aip_req','dir_exd','sub_dir_exd',
'jer_exd','niv_mod','gestion','d_gestion','ges_dep','d_ges_dep','estado',
'd_estado','d_region','tipo_entidad','jec','Unnamed: 0','year','otro_doc_exd','d_dpto','codlocal','d_dist'])

In [32]:
# Other outcomes not relevant to predict in this version
other_outcomes=['doc_aula_req','aux_req','doc_aula_exd_n', 'doc_aula_exd_c', 'aux_exd']
df = df.drop(columns=other_outcomes)

# Feature storage 

## Numbers variables

In [33]:
workers = ['dir_n','dir_c','sub_dir_n','sub_dir_c', 'doc_aula_n', 'doc_aula_c', 'otro_doc_n', 'otro_doc_c', 'aux_n', 'aux_c']

In [34]:
enrollment = [x for x in df.columns.to_list() if (x.find('cant')!=-1) or (x.find('inclu')!=-1)]

In [35]:
gps = ['nlat_ie','nlong_ie','ubigeo','codooii']

In [36]:
numerical_inputs = workers+enrollment+gps

In [37]:
workers_stats = df[workers].describe().transpose()

In [38]:
workers_stats['cv'] = workers_stats.iloc[:,2] / workers_stats.iloc[:,1] * 100
workers_stats

,count,mean,std,min,25%,50%,75%,max,cv
dir_n,160087.0,0.131222,0.337644,0.0,0.0,0.0,0.0,1.0,257.307046
dir_c,160087.0,0.049904,0.217748,0.0,0.0,0.0,0.0,1.0,436.331832
sub_dir_n,160087.0,0.022769,0.151782,0.0,0.0,0.0,0.0,2.0,666.619100
sub_dir_c,160087.0,0.013187,0.131522,0.0,0.0,0.0,0.0,5.0,997.390957
doc_aula_n,160087.0,2.332925,4.473909,0.0,0.0,1.0,2.0,66.0,191.772494
doc_aula_c,160087.0,1.055745,1.484930,0.0,0.0,1.0,1.0,27.0,140.652410
otro_doc_n,160087.0,0.058543,0.342958,0.0,0.0,0.0,0.0,7.0,585.820524
otro_doc_c,160087.0,0.125319,0.478971,0.0,0.0,0.0,0.0,8.0,382.200324
aux_n,160087.0,0.063291,0.404233,0.0,0.0,0.0,0.0,12.0,638.693678
aux_c,160087.0,0.150443,0.594403,0.0,0.0,0.0,0.0,15.0,395.101118


## Categorical variables

In [39]:
categorical=['d_niv_mod', 'ruralidad', 'vraem', 'frontera', 'bilingue',
 'caracteristica','d_prov','d_dreugel']

# Train/Test split

In [40]:
inputs = numerical_inputs + categorical

In [41]:
X = df[inputs]
y = df[['secciones_necesarias']]

In [42]:
from sklearn.model_selection import train_test_split

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state=36936330)

In [45]:
# Export train and test data
# Train
X_train.to_csv(work/r'Results\\X_train.csv')
y_train.to_csv(work/r'Results\\y_train.csv')

#Test
X_test.to_csv(work/r'Results\\X_test.csv')
y_test.to_csv(work/r'Results\\y_test.csv')